<a href="https://colab.research.google.com/github/k4karthi/Indian-Sign-Language-to-Text-Conversion/blob/main/mobilenetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/AssemblyAI-Examples/mediapipe-python.git
!pip install mediapipe
!pip install PyQt5
!pip install ipython==7.32.0
!pip install albumentations

In [ ]:
import os
import cv2
import random
import pickle
import warnings
import numpy as np
import pandas as pd
import urllib.request
from PIL import Image
from base64 import b64encode
from tqdm import tqdm
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib as mpl
import importlib
import PyQt5
import mediapipe as mp
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from keras.src import backend
from keras.src import layers
from keras.src.api_export import keras_export
from keras.src.applications import imagenet_utils
from keras.src.models import Functional
from keras.src.ops import operation_utils
from keras.src.utils import file_utils


##Data Augmentation

In [ ]:

# Define augmentation pipeline
augmentation = A.Compose([
    A.Rotate(limit=5, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.4),
    A.HueSaturationValue(hue_shift_limit=2, sat_shift_limit=5, val_shift_limit=2, p=0.3),
    A.MotionBlur(blur_limit=3, p=0.2),
    A.GaussianBlur(blur_limit=(3, 3), p=0.1),
    A.CLAHE(clip_limit=1.0, tile_grid_size=(8, 8), p=0.2),
])

# Set paths
input_dir = "path to input directory"
output_dir = "path to output directory"
os.makedirs(output_dir, exist_ok=True)

# Randomly select n classes from the dataset. Change the 'n'

n=10

all_classes = [folder for folder in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, folder))]
selected_classes = random.sample(all_classes, n)

# Number of images per class . change as per your need
target_count = 1000

for class_folder in tqdm(selected_classes):
    class_path = os.path.join(input_dir, class_folder)
    output_class_path = os.path.join(output_dir, class_folder)
    os.makedirs(output_class_path, exist_ok=True)

    images = [img for img in os.listdir(class_path) if img.lower().endswith((".jpg", ".png", ".PNG"))]
    original_count = len(images)

    # Copy original images first
    for img_name in images:
        img_path = os.path.join(class_path, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(output_class_path, img_name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    # Augment until reaching 750 images
    augment_needed = target_count - original_count
    if augment_needed > 0:
        for i in range(augment_needed):
            img_name = random.choice(images)
            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            augmented = augmentation(image=image)["image"]
            aug_img_name = f"{img_name.split('.')[0]}_aug{i}.jpg"
            cv2.imwrite(os.path.join(output_class_path, aug_img_name), cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))


print(f"Augmentation completed for {n} classes with {target_count} images each.")


##Bounding Box Creation using Mediapipe

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

In [ ]:
data_dir = 'path to dataset'  # Your dataset path (15 gesture folders)
img_size = 224  # Required size for MobileNetV2
data = []
labels = []
classes = sorted(os.listdir(data_dir))


In [ ]:
for label in tqdm(classes):
    if label == '.DS_Store':
        continue
    label_path = os.path.join(data_dir, label)
    for img_name in os.listdir(label_path):
        img_path = os.path.join(label_path, img_name)
        image = cv2.imread(img_path)

        if image is None:
            continue

        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                h, w, _ = image.shape
                x_vals = [lm.x * w for lm in hand_landmarks.landmark]
                y_vals = [lm.y * h for lm in hand_landmarks.landmark]

                x_min, x_max = int(min(x_vals)) - 20, int(max(x_vals)) + 20
                y_min, y_max = int(min(y_vals)) - 20, int(max(y_vals)) + 20

                x_min = max(0, x_min)
                y_min = max(0, y_min)
                x_max = min(w, x_max)
                y_max = min(h, y_max)

                hand_img = img_rgb[y_min:y_max, x_min:x_max]
                if hand_img.size == 0:
                    continue

                hand_img = cv2.resize(hand_img, (img_size, img_size))
                hand_img = preprocess_input(hand_img)  # MobileNetV2 preprocessing
                data.append(hand_img)
                labels.append(classes.index(label))


##Train-Test Split

In [ ]:
X = np.array(data)
y = to_categorical(labels, num_classes=len(classes))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=22, shuffle=True)


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/processed_dataset",
    validation_split=0.15,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/processed_dataset",
    validation_split=0.15,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)


##Model Compilation & Training

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(img_size, img_size, 3),
                                               include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


##Model Serialization

In [ ]:
model.save('mobilenetv2_handgesture_model.h5')
